In [2]:
import pandas as pd
labeller = pd.read_csv('tenSeq.csv', sep=',',header=0)
# labeller = pd.read_csv('rnn_final.csv', sep=',',header=0)
labeller


,1,sequence
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
5,1,6
6,1,7
7,1,8
8,1,9
9,1,10


In [3]:
import sys
import random
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from tqdm import tqdm
from tensorflow.python.ops import rnn, rnn_cell


In [4]:
path = "/home/treav/Desktop/production/FinalProject/"
# Setting working directory
os.chdir(path)

my_data = np.matrix(np.genfromtxt('tenSeq.csv', delimiter=',' , dtype=str , skip_header=1 ))
# my_data = np.matrix(np.genfromtxt('rnn_final.csv', delimiter=',' , dtype=str , skip_header=1 ))




d = "/home/treav/Desktop/production/FinalProject/melodies"
# Set directory for wav files
os.chdir(d)

In [5]:
l={}
r = []
name = []
dir_length=len(os.listdir(os.getcwd()))
print("\nImporting wav files...")
#pbar = tqdm(total=dir_length) # Specify the progressBar
for filename in tqdm(os.listdir(os.getcwd())):    
    x,sr = librosa.load(filename)    
    l[filename] = list()
    l[filename].append(x)
    r.append(x)
    name.append(filename.split('.')[0])

# Make raw as an np.array
raw = np.array(r)    

  0%|          | 0/20 [00:00<?, ?it/s]


Importing wav files...


100%|██████████| 20/20 [05:47<00:00, 17.35s/it]
<ipython-input-5-e20f28d1555a>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw = np.array(r)


In [72]:
ind = np.where(my_data[:,0] == name)[0] # find indexes of our wav in my_data
truth = np.matrix(my_data[ind]) # make truth only with these indexes
# Fix the order of truth
tmp = np.argsort(np.where(truth[:,0] == name)[1]) # index the mapping 
truth = truth[tmp] # set the actural order



In [73]:
random.seed(1)
all_ind = np.where(truth[:,0])[0]
# Make train set contain 70% random observations of data
tr_ind = random.sample(list(all_ind) , round(70/100 * len(raw) ) )
#
tr_sound , tr_truth = raw[tr_ind] , truth[tr_ind]  
ts_sound , ts_truth = raw[np.delete(all_ind , tr_ind)] , truth[np.delete(all_ind , tr_ind)]


In [74]:
##########
# Extract mfcc feature by windowing for RNN
# Windowing
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

# Function for mel-frequency-spectogram-coefficinets with windowing for RNN
def extract_mfcc(sound, groundTruth, bands = 20, frames = 41):
    window_size = 512 * (frames - 1)
    mfccs = []
    labels = np.empty(0)
    for wav in tqdm(sound):        
        for (start,end) ,lab in zip( windows(wav , window_size) , groundTruth[:,1] ):
            if(len(wav[int(start):int(end)]) == window_size):
                signal = wav[int(start):int(end)]
                mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                mfccs.append(mfcc)
                labels = np.append(labels , lab)         
    features = np.asarray(mfccs).reshape(len(mfccs),frames,bands)
    return np.array(features) , np.array(labels,dtype = np.int)




In [75]:
""" Define the target hot_vector (bitmap) 2-d array col1 = 1 and col2 = 0 if NOT bird
 - Input: array of labels
 - Output: two column array with 1 where is true """
 
def hot_vector(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

    

In [76]:
# Make the input for the NN
print("\nConstructing the train_set...")
train_x , train_lab = extract_mfcc(tr_sound , tr_truth)
print("\nDone >_ \n")
print("Constructing the test_set...")
test_x , test_lab = extract_mfcc(ts_sound , ts_truth)
print("\nDone >_ \n")    


  0%|          | 0/14 [00:00<?, ?it/s]


Constructing the train_set...


100%|██████████| 14/14 [00:07<00:00,  1.89it/s]
<ipython-input-74-384043ac6cda>:23: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(features) , np.array(labels,dtype = np.int)
  0%|          | 0/6 [00:00<?, ?it/s]


Done >_ 

Constructing the test_set...


100%|██████████| 6/6 [00:00<00:00,  9.64it/s]


Done >_ 



In [77]:
# Make hotvector of labels
tr_labels = hot_vector(train_lab)
ts_labels = hot_vector(test_lab)

IndexError: index 15 is out of bounds for axis 1 with size 14

## NEW WAY DIY


In [80]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt



In [5]:
from glob import glob
norm_data_dir = '/home/treav/Desktop/production/FinalProject/melodies'
norm_audio_files = glob(norm_data_dir + '*.mp3')


with open(norm_data_dir, "r") as fp:
    data = json.load(fp)

print(data)


IsADirectoryError: [Errno 21] Is a directory: '/home/treav/Desktop/production/FinalProject/melodies'

In [ ]:

def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y



def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test
